In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import data from csv
df = pd.read_csv('notion_export.csv')
df.head()

,Doc Name,Date,Status,Proposal,Type of Doc,Actor,Technical Field,Objectives and Priorities,Cyber Capabilities,Roles and Responsabilities,Resource Allocation,Challenges and Threats,Link
0,"Resilience, Deterrence and Defence: Building s...",13/09/2017,Not started,No,Communication,EU Comm,JOIN(2017) 450 final,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...
1,concerning restrictive measures against cyber-...,17/05/2019,Not started,No,Decision,EU Council,L 129 I/13,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...
2,Cybersecurity Act,07/06/2019,Not started,No,Regulation,Legislative,526/2013,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...
3,The EU's Cybersecurity Strategy for the Digita...,16/12/2020,Not started,No,Communication,EU Comm,JOIN(2020) 18 final,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...
4,The EU's Cybersecurity Strategy for the Digita...,16/12/2020,Not started,No,Communication,EU Comm,JOIN(2020) 18 final,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...


In [63]:
from urllib.parse import urlparse, parse_qs

# Ottieni l'URL dal dataframe
url = df['Link']

# Analizza l'URL
parsed_urls = url.apply(urlparse)

# Ottieni il parametro 'uri'
uri_params = parsed_urls.apply(lambda x: parse_qs(x.query).get('uri', [''])[0])

# Substituisci i caratteri speciali
uri_params = uri_params.str.replace(':', '_')

print(uri_params)

0                                     CELEX_52017JC0450
1                                      CELEX_32019D0797
2                                      CELEX_32019R0881
3                                     CELEX_52020JC0018
4                                     CELEX_52020JC0018
5                                      CELEX_32021R0697
6                                                      
7                                                      
8     cellar_864f472b-34e9-11ed-9c68-01aa75ed71a1.00...
9                                                      
10                                                     
11                                     CELEX_32022L2555
12                                     CELEX_32022L2557
13                                                     
14                                                     
15                                     CELEX_32021H1086
Name: Link, dtype: object


In [65]:
# In case of Celex absence, insert External_1 or External_2 
if pd.isnull(df['Celex']).any():
    celex = df['External_1']
else:
    celex = df['Celex']

In [72]:
from urllib.parse import urlparse, parse_qs

def get_uri_param(row):
    url = row['Link']
    parsed_url = urlparse(url)
    uri_param = parse_qs(parsed_url.query).get('uri', [None])[0]
    
    # Sostituisci i caratteri speciali
    if uri_param:
        uri_param = uri_param.replace(':', '_')
    
    # In caso di assenza di uri_param, inserisci External_1 o External_2
    if not uri_param or uri_param == 'SomeInvalidValue':
        if 'External_1' in row and row['External_1']:
            uri_param = row['External_1']
        elif 'External_2' in row and row['External_2']:
            uri_param = row['External_2']
        else:
            uri_param = ''
    
    return uri_param

# Usa la funzione per assegnare a ciascuna riga il proprio 'uri_param'
df['Celex'] = df.apply(get_uri_param, axis=1)
df.head()

,Doc Name,Date,Status,Proposal,Type of Doc,Actor,Technical Field,Objectives and Priorities,Cyber Capabilities,Roles and Responsabilities,Resource Allocation,Challenges and Threats,Link,Celex
0,"Resilience, Deterrence and Defence: Building s...",13/09/2017,Not started,No,Communication,EU Comm,JOIN(2017) 450 final,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,CELEX_52017JC0450
1,concerning restrictive measures against cyber-...,17/05/2019,Not started,No,Decision,EU Council,L 129 I/13,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,CELEX_32019D0797
2,Cybersecurity Act,07/06/2019,Not started,No,Regulation,Legislative,526/2013,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,CELEX_32019R0881
3,The EU's Cybersecurity Strategy for the Digita...,16/12/2020,Not started,No,Communication,EU Comm,JOIN(2020) 18 final,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,CELEX_52020JC0018
4,The EU's Cybersecurity Strategy for the Digita...,16/12/2020,Not started,No,Communication,EU Comm,JOIN(2020) 18 final,NaN,NaN,NaN,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,CELEX_52020JC0018


In [80]:
import os
import requests


# Crea una lista di tutti i link
links = df['Link'].tolist()



# Scarica tutti i PDF con il CELEX come nome del file e li salva in una nuova cartella chiamata "EU Docs"
folder_name = "EU Docs"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i, link in enumerate(links):
    response = requests.get(link, stream=True)
    # Usa l'indice come nome del file
    filename = str(i)
    filepath = os.path.join(folder_name, filename)
    with open(filepath, 'wb') as f:
        f.write(response.content)
    if not os.path.exists(filepath):
        print(f"File {filepath} does not exist")
    else:
        print(f"File {filepath} downloaded successfully")

File EU Docs\0 downloaded successfully
File EU Docs\1 downloaded successfully
File EU Docs\2 downloaded successfully
File EU Docs\3 downloaded successfully
File EU Docs\4 downloaded successfully
File EU Docs\5 downloaded successfully
File EU Docs\6 downloaded successfully
File EU Docs\7 downloaded successfully
File EU Docs\8 downloaded successfully
File EU Docs\9 downloaded successfully
File EU Docs\10 downloaded successfully
File EU Docs\11 downloaded successfully
File EU Docs\12 downloaded successfully
File EU Docs\13 downloaded successfully
File EU Docs\14 downloaded successfully
File EU Docs\15 downloaded successfully
